In [2]:
# import data
import sys
import pickle
import re
import codecs
import string
import shutil
from win32com import client as wc
import docx

import os
dirPath = r"C:\Users\traffic\Desktop\package\data"
dirPath2 = r'C:\Users\traffic\Desktop\0613網路發件'

# result = [f for f in os.listdir(dirPath) if os.path.isfile(os.path.join(dirPath, f))]
# print(result)

from os import listdir
from os.path import isfile, isdir, join
files = listdir(dirPath)

## change doc into docx
for i in files:
    try:
        word = wc.Dispatch('Word.Application')
        doc = word.Documents.Open(u'C:\\Users\\traffic\\Desktop\\package\\data\\' + i + '\\' + i + '.doc')        # 目標路徑下的檔案
        doc.SaveAs(u'C:\\Users\\traffic\\Desktop\\package\\data\\' + i + '\\' + i + '.docx', 12, False, "", True, "", False, False, False, False)  # 轉化後路徑下的檔案    
        doc.Close()
    except:
        pass
    #word.Quit()
    
date_list = []
car_list = []
data = []

for k,i in enumerate(files):
    #獲取文件物件
    file=docx.Document("C:\\Users\\traffic\\Desktop\\package\\data\\" + i +'\\' + i +".docx")
    #print("段落數:"+str(len(file.paragraphs)))#段落數為13，每個回車隔離一段

    car_num = []

    #輸出每一段的內容
    #for para in file.paragraphs:
    #   print(para.text)

    #輸出段落編號及段落內容
    for j in range(len(file.paragraphs)):
        #print("第"+str(i)+"段的內容是："+file.paragraphs[i].text)
        if file.paragraphs[j].text.split(':')[0] != '違規車號' and file.paragraphs[j].text.split(':')[0] != '\n違規車號':
            continue
        else:
            car_num.append(j)
    if len(car_num) != 0:
        date_list.append(file.paragraphs[car_num[0]+1].text.split(':')[1].split('日')[0].replace('年','0').replace('月','')[1:])
    else:
        date_list.append('')
    car = ''  
    for i in car_num:
        car += file.paragraphs[i].text.split(':')[1] + str('、')
    car[:-1]
    car_list.append(car[:-1])
    
    i = files[k]
    file=docx.Document("C:\\Users\\traffic\\Desktop\\package\\data\\" + i +'\\' + i +".docx")
    #print(k)
#     try:
#         table = file.tables[1]
#         keys = None
#         for i, row in enumerate(table.rows):
#             text = (cell.text for cell in row.cells)

#             # Establish the mapping based on the first row
#             # headers; these will become the keys of our dictionary
#             if i == 0:
#                 keys = tuple(text)
#                 continue

#             # Construct a dictionary for this row, mapping
#             # keys to values for this row
#             row_data = dict(zip(keys, text)) 
#             if not row_data['回復內文']=='第 二 層決行' and not row_data['回復內文']=='交通隊            核稿               決行\n\xa0\n\xa0\n\xa0\xa0':
#                 data.append(row_data['回復內文'])
#     except KeyError:
#         try:
#             table = file.tables[1].rows[0].cells[1].text
#             data.append(table)
#         except:
#             table = file.tables[0].rows[1].cells[0].text
#             data.append(table)
            
import pandas as pd
from pandas import Series,DataFrame
#data_s = Series(data,name = 'data',index = files)
date_s = Series(date_list,name = 'date',index = files)
car_s = Series(car_list,name = 'car',index = files)
# df = pd.concat([date_s,car_s,data_s],axis = 1)
df = pd.concat([date_s,car_s],axis = 1)

import xlsxwriter

workbook = xlsxwriter.Workbook('Hyperlinks(開完記得刪掉).xlsx')
worksheet = workbook.add_worksheet('Hyperlinks')

month = input('Enter a month : ')
day = input('Enter a day : ')

for i,f in enumerate(files,start=1):   
    worksheet.write_url('A'+str(i), '108結畢案件\\108年' + month + '月'+ '\\' + day + '\\'+ f,string = f)
    worksheet.write('B'+str(i),date_s[f])
    worksheet.write('C'+str(i),car_s[f])
    #worksheet.write('D'+str(i),data_s[f])
workbook.close()

Enter a month : 7
Enter a day : 0716
